In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
import numpy as np


img_shape = (224, 224, 3)
img_input = Input(shape=img_shape)

base_model = InceptionV3(weights='imagenet', input_tensor=img_input)

for layer in base_model.layers[:-6]:
    layer.trainable=False

net = Flatten()(base_model.layers[-3].output)
net = Dense(1024, activation='relu')(net)
net = Dropout(0.2)(net)
dogcat = Dense(1, activation="sigmoid")(net)


img_path = 'krzych.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(base_model.layers[0])
#in_img = model.layers[0].inputs


model = Model(inputs=[img_input], outputs=[dogcat])

preds = model.predict(x)

print("Predictions : ", preds)

model.summary()

Predictions :  [[0.3122734]]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_188 (Bat (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_v1_188[0][0] 
________________________________________________________________________________

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size = 20

train_generator = train_datagen.flow_from_directory("./training", batch_size=batch_size, class_mode='binary', target_size=(224, 224))

validation_generator = test_datagen.flow_from_directory("./validation", batch_size=batch_size, class_mode='binary', target_size=(224, 224))

#model2.load_weights("transfer2.h5")

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit_generator(train_generator, 
                              validation_data=validation_generator, 
                              steps_per_epoch=100, 
                              epochs=10, 
                              validation_steps=50,
                              verbose=2)

Found 22496 images belonging to 2 classes.
Found 2702 images belonging to 2 classes.
Epoch 1/10
136/136 [==============================] - 12s 86ms/step - loss: 0.5171 - acc: 0.9660
 - 145s - loss: 4.5454 - acc: 0.7115 - val_loss: 0.5171 - val_acc: 0.9660
Epoch 2/10
136/136 [==============================] - 11s 79ms/step - loss: 0.7655 - acc: 0.9515
 - 144s - loss: 1.8343 - acc: 0.8830 - val_loss: 0.7655 - val_acc: 0.9515
Epoch 3/10
136/136 [==============================] - 11s 80ms/step - loss: 0.5747 - acc: 0.9637
 - 143s - loss: 1.7192 - acc: 0.8907 - val_loss: 0.5747 - val_acc: 0.9637
Epoch 4/10
136/136 [==============================] - 11s 79ms/step - loss: 0.4082 - acc: 0.9745
 - 143s - loss: 1.6379 - acc: 0.8962 - val_loss: 0.4082 - val_acc: 0.9745
Epoch 5/10
136/136 [==============================] - 11s 80ms/step - loss: 0.4832 - acc: 0.9730
 - 143s - loss: 1.4972 - acc: 0.9053 - val_loss: 0.4832 - val_acc: 0.9730
Epoch 6/10
136/136 [==============================] - 11s 79

In [8]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
import matplotlib    
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

acc      = history.history[     'acc' ]
val_acc  = history.history[ 'val_acc' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

Text(0.5, 1.0, 'Training and validation loss')